<a href="https://colab.research.google.com/github/hyeonniii/NLP/blob/main/0812%EA%B5%90%EC%9C%A13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

text = "국책연구기관 딥러닝 기본과정입니다."
text_to_word_sequence(text)


['국책연구기관', '딥러닝', '기본과정입니다']

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer

doc = ['국책연구기관 딥러닝 기본과정입니다.','한국개발연구원 정책대학원 컴퓨터실입니다.', '딥러닝 마스터가 됩시다.']
token = Tokenizer()
token.fit_on_texts(doc)
print(token.word_counts)

OrderedDict([('국책연구기관', 1), ('딥러닝', 2), ('기본과정입니다', 1), ('한국개발연구원', 1), ('정책대학원', 1), ('컴퓨터실입니다', 1), ('마스터가', 1), ('됩시다', 1)])


## 네이버 영화 리뷰 데이터를 활용한 영화 감성분석 예제


### Initialization

In [6]:
# 최신 버전의 사이킷런을 설치 및 확인
!pip install --upgrade scikit-learn
# 한국형 형태소 분석(표제어 추출 방식)을 위한 패키지 'konlpy'를 설치
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.4 MB/s 
     |████████████████████████████████| 453 kB 40.4 MB/s 


In [7]:
"""
필요한 함수 불러오기:
konlpy - ("코엔엘파이"라고 읽습니다)는 한국어 정보처리를 위한 파이썬 패키지 입니다 
(https://konlpy.org/ko/v0.5.2/)
konlpy는 한국어 형태소 분석기 패키지 입니다
"""

import konlpy
import pandas as pd
import numpy as np
import os

In [8]:
"""
- 깃허브에 있는 데이터를 빌려(?) 오기
- 데이터는 IMDb의 영화 리뷰 데이터셋과 비슷한 네이버 영화 리뷰 데이터셋(https://github.com/e9t/nsmc)을 활용하였습니다
- 약 20만건의 영화 리뷰를 2015년을 기준으로 수집한 데이터 입니다.
- "머신러닝 교과서 with 파이썬, 사이킷런, 텐서프로 (개정3판)"에서 해당 데이터를 따로 제공하고 있으므로, 해당 깃허브를 통해서 간단히 내려받기 합니다
"""

# Training (훈련) 데이터 받기
!wget https://github.com/rickiepark/python-machine-learning-book-3rd-edition/raw/master/ch08/ratings_train.txt -O ratings_train.txt

# Testing (테스트) 데이터 받기
!wget https://github.com/rickiepark/python-machine-learning-book-3rd-edition/raw/master/ch08/ratings_test.txt -O ratings_test.txt

--2022-08-12 05:17:42--  https://github.com/rickiepark/python-machine-learning-book-3rd-edition/raw/master/ch08/ratings_train.txt
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rickiepark/python-machine-learning-book-3rd-edition/master/ch08/ratings_train.txt [following]
--2022-08-12 05:17:42--  https://raw.githubusercontent.com/rickiepark/python-machine-learning-book-3rd-edition/master/ch08/ratings_train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: ‘ratings_train.txt’

ratings_train.txt   100%[===================>]  13.95M  --.-KB/s    in 0.1s    



### Data preprocessing


In [10]:
# 내려받기 된 데이터를 읽어볼까요?
"""
(부록 F 참조)
+-----------------------+
|     Training Set     |  <--- 약 75%
+-----------------------+
|      Testing Set     |  <--- 약 25%
+-----------------------+

"""
# Training Data
Review_train = pd.read_csv('ratings_train.txt', 
                       delimiter='\t', keep_default_na=False)

# Testing Data
Review_test = pd.read_csv('ratings_test.txt', 
                      delimiter='\t', keep_default_na=False)

# 데이터의 첫 5줄 정도 빠르게 보기를 통한 내용 파악
# document 컬럼은 리뷰내용을, label은 긍정(1)과 부정(0)을 나타내고 있습니다
# training data 확인

Review_train.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [11]:
!wget https://github.com/ycanns/DeepLearning_Class_NRC/blob/f6b7102641d4580ec788f29157eff410a681042e/data/naver_data.zip

--2022-08-12 05:37:00--  https://github.com/ycanns/DeepLearning_Class_NRC/blob/f6b7102641d4580ec788f29157eff410a681042e/data/naver_data.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘naver_data.zip’

naver_data.zip          [ <=>                ] 139.74K  --.-KB/s    in 0.08s   

2022-08-12 05:37:00 (1.79 MB/s) - ‘naver_data.zip’ saved [143094]



In [18]:
!unzip -qq "/content/naver_data.zip" -d "/content/naver_data"

[/content/naver_data.zip]
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/naver_data.zip or
        /content/naver_data.zip.zip, and cannot find /content/naver_data.zip.ZIP, period.


In [19]:
# 데이터 그룹(document, 리뷰내용)과 타겟데이터 그룹 (label, 긍/부정)으로 분류하여 저장
"""
(부록 F 참조)
    (document)      (label)
+----------------+-------------+
|   Data_train   | Target_train|  <--- Training Set (Review_train)
+------------------------------+
|    Data_test   | Target_test |  <--- Testing Set (Review_test)
+------------------------------+

"""
# Review_train 데이터의 훈련데이터 
Data_train = Review_train['document'].values
Target_train = Review_train['label'].values

# Review_test 데이터의 훈련데이터 
Data_test = Review_test['document'].values
Target_test = Review_test['label'].values

# training 데이터의 크기확인
print(len(Data_train), np.bincount(Target_train))

# testing 데이터의 크기확인
print(len(Data_test), np.bincount(Target_test))

150000 [75173 74827]
50000 [24827 25173]


### Data analysis & training

In [20]:
from konlpy.tag import Okt

okt = Okt() # Okt 형태소 분석기 호출

# 4번째 리뷰 내용을 바탕으로 형태소분리 (morphs 행태소 추출 모듈)
print(Data_train[4])
print(okt.morphs(Data_train[4]))
print(okt.nouns(Data_train[4])) # 명사추출 예시

사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
['사이', '몬페', '그', '의', '익살스런', '연기', '가', '돋보였던', '영화', '!', '스파이더맨', '에서', '늙어', '보이기만', '했던', '커스틴', '던스트', '가', '너무나도', '이뻐', '보였다']
['몬페', '의', '연기', '영화', '스파이더맨', '커스틴', '던스트']


In [22]:
from scipy.sparse import save_npz, load_npz
from sklearn.feature_extraction.text import TfidfVectorizer

if not os.path.isfile('okt_train.npz'):
    """
    - TfidVectorzier는 기본적으로 공백을 기준으로 토큰을 구분
    - ngram_range : N-gram에서 고려할 단어뭉치 수 (1개 유니그램, 2개 바이그램)
    - min_df : 최소 고려 토큰 (특정 횟수이상 등장하는 토큰만을 고려)
    - max_df : 무시할 가장 많이 등장하는 상위 백분율
    - okt.morphs: 형태소 분석을 통한 토큰화
    """ 

    tfidf = TfidfVectorizer(ngram_range=(1, 2), 
                            min_df=3,
                            max_df=0.9,
                            tokenizer=okt.morphs, 
                            token_pattern=None)
    tfidf.fit(Data_train)
    Data_train_okt = tfidf.transform(Data_train)
    Data_test_okt = tfidf.transform(Data_test)
    save_npz('okt_train.npz', Data_train_okt)
    save_npz('okt_test.npz', Data_test_okt)
else:
    Data_train_okt = load_npz('okt_train.npz')
    Data_test_okt = load_npz('okt_test.npz')

### Run model

In [23]:
# 파라메터 최적화를 위한 분류기법 등의 모듈 불러오기
from sklearn.model_selection import RandomizedSearchCV 
from sklearn.linear_model import SGDClassifier # Stochastic Gradient Descent (SGD)
from sklearn.utils.fixes import loguniform



#- 만약, 가용한 CPU 코어의 여유가 있다면 n_jobs (병렬 처리 할 때 사용되는 CPU 코어 수)를 추가하여 설정
sgd = SGDClassifier(loss='log', random_state=1) 
param_dist = {'alpha': loguniform(0.0001, 100.0)} # alpha : 값이 클수록 강력한 정규화(규제) 설정

rsv_okt = RandomizedSearchCV(estimator=sgd,
                             param_distributions=param_dist,
                             n_iter=50,
                             random_state=1,
                             verbose=1) #0 = 아무것도 표시 않음/ 1 = 프로그래스 바 표시/ 2 = 이포크마다 표시
rsv_okt.fit(Data_train_okt, Target_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


RandomizedSearchCV(estimator=SGDClassifier(loss='log', random_state=1),
                   n_iter=50,
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f96f22e2110>},
                   random_state=1, verbose=1)

### Results

In [ ]:
print(rsv_okt.best_score_)  # 최적의 실험결과
print(rsv_okt.best_params_) # 최적의 파라메터 값 (alpha)
# 테스트셋을 이용한 검증
rsv_okt.score(Data_test_okt, Target_test)

## IMDb 영화 리뷰 데이터를 이용한 감성분석 + 머신러닝 예시


### Initialization

In [24]:
# 최신 버전의 사이킷런을 설치 및 확인
!pip install --upgrade scikit-learn


import os
import sys
import tarfile        # tar 형태의 압축파일 해제를 위한 모듈
import time           # 시간 확인을 위한 모듈 (옵션)
import urllib.request # 인터넷 주소로부터 데이터를 구득하기 위한 모듈

# 데이터를 불러올 소스의 인터넷 주소와 파일명 (직접 받아도 되고.. )
source = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
target = 'aclImdb_v1.tar.gz' # 해당 파일이름


def reporthook(count, block_size, total_size):
    global start_time
    if count == 0:
        start_time = time.time()
        return
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = progress_size / (1024.**2 * duration)
    percent = count * block_size * 100. / total_size

    sys.stdout.write("\r%d%% | %d MB | %.2f MB/s | %d sec elapsed" %
                    (percent, progress_size / (1024.**2), speed, duration))
    sys.stdout.flush()


if not os.path.isdir('aclImdb') and not os.path.isfile('aclImdb_v1.tar.gz'):
    urllib.request.urlretrieve(source, target, reporthook)
    

# Download가 완료된 tar 파일의 압축 해제
if not os.path.isdir('aclImdb'):

    with tarfile.open(target, 'r:gz') as tar:
        tar.extractall()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
100% | 80 MB | 1.88 MB/s | 42 sec elapsed

In [25]:
""" 
필요한 함수들을 불러오기: 
numpy - 복잡한 산술계산,
padas - 데이터의 효과적인 연산을 위한 모듈
"""

import pandas as pd
import numpy as np
import os

# `basepath`를 압축 해제된 영화 리뷰 데이터셋이 있는 디렉토리로 바꿉니다
basepath = 'aclImdb'

# 폴더명을 기준으로 각 txt 파일을 읽고, 리뷰평을 추가하는 방식
# 별점 5개는 긍정/ 5개 미만은 부정으로 기록이 되어있습니다

labels = {'pos': 1, 'neg': 0}
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file), 
                      'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], 
                           ignore_index=True)
df.columns = ['review', 'sentiment']

### Data Preprocessing

In [ ]:
# 한데 모아진 영화 리뷰 데이터의 순서를 섞습니다.
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.head()

In [ ]:
# 불필요한 특수문자나 HTML 언어 등을 선별하여 정리
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text)
    text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emoticons).replace('-', ''))
    return text

df['review'] = df['review'].apply(preprocessor)
df.head()

In [ ]:
# 훈련 (약 75%) 데이터와 테스트 (약 25%) 데이터로 나누어 저장
X_train = df.loc[:37500, 'review'].values
y_train = df.loc[:37500, 'sentiment'].values
X_test = df.loc[17500:, 'review'].values
y_test = df.loc[17500:, 'sentiment'].values

### Working on Texts

In [ ]:
# 문장들을 토큰으로 나누기
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer(text):
    return text.split() # 공백을 기준으로 나누기

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()] # 어간추출방법을 이용

In [ ]:
# 공백기준 토큰나누기 예시
tokenizer("runners like running and thus they run")

In [ ]:
# 어간추출기준 토큰나누기 예시
tokenizer_porter("runners like running and thus they run")

In [ ]:
# 불용어 집합을 불러오기
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop = stopwords.words('english')